In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow as tf

import shutil

from tqdm import tqdm

In [2]:
# Define your image path
folder_path = '/Users/dheemankumar/github/pc-clener/folder'
# Load your trained model
model = load_model('/Users/dheemankumar/github/pc-clener/segmentor/your_model.h5')


2024-03-21 12:15:29.216994: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-03-21 12:15:29.217018: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-03-21 12:15:29.217022: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-03-21 12:15:29.217052: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-21 12:15:29.217067: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
print(tf.__version__)

2.14.0


In [4]:
labels=['docks', 'object', 'persons', 'place']

In [5]:
def append_to_text_file(image, output_file):
    with open(output_file, 'a') as f:  # 'a' mode for append
        f.write(image + '\n')


In [6]:
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    image = cv2.resize(image, (224, 224))  # Resize to match input size of your model
    image = image / 255.0  # Normalize pixel values to [0, 1]
    return image

In [7]:
# Define a function to predict the label of an image
def predict_image_label(image_path):
    image = preprocess_image(image_path)
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    prediction = model.predict(image,verbose=0)
    predicted_class = np.argmax(prediction)
    return predicted_class


In [8]:
def moveimage(image_path):
    predicted_label = predict_image_label(image_path)
    
    shutil.move(image_path,labels[predicted_label])

In [9]:
def list_images(folder_path):

    # Get the total number of files in the directory tree
    total_files = sum(len(files) for _, _, files in os.walk(folder_path))
    
    # Initialize tqdm with the total number of files
    progress_bar = tqdm(total=total_files, desc='Processing images', unit='files')
    
    
    # Walk through the directory tree
    for root, dirs, files in os.walk(folder_path):
        # Iterate through files
        for file in files:
            # Check if the file has an image extension
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                # Construct the full path to the image file
                image_path = os.path.join(root, file)
                moveimage(image_path)
                # append_to_text_file(image_path, 'image_list.txt')

                # Update the progress bar
                progress_bar.update(1)
    
    progress_bar.close()

In [10]:
for label in labels:
    os.makedirs(label, exist_ok=True)


image_list = list_images(folder_path)


Processing images:   0%|          | 0/1629 [00:00<?, ?files/s]

2024-03-21 12:15:30.391395: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
Processing images: 100%|█████████▉| 1628/1629 [02:39<00:00, 10.21files/s]
